## The idea of this exercise

* The idea of this exercise is to build a simple RAG pipeline with text intensive data
* We would use vertex embedding 
* we would use FIASS vector store
* we would create a prompt for a student studying class 10 clarifying some doubts


In [1]:
# indexing pipeline

# indexing
# 1. loader
from langchain_community.document_loaders import DirectoryLoader, TextLoader
# 2. splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
# 3. embeddings
from langchain_google_vertexai.embeddings import VertexAIEmbeddings
# 4. vectorstore
from langchain_community.vectorstores import FAISS

In [2]:
# lets load documents

directory_loader = DirectoryLoader(
    path="../data/concepts",
    glob="*.txt",
    loader_cls=TextLoader,
)
documents = directory_loader.load()

In [6]:
# investigate documents
print(f"number of docs {len(documents)}")
print("\n\n*****************\n\n")
print(f"100 characters content of first doc \n{documents[0].page_content[100:]}")
print("\n\n*****************\n\n")
print(f"metadata of first doc \n{documents[0].metadata}")


number of docs 6


*****************


100 characters content of first doc 
celerates when it speeds up, slows down, or changes direction. Acceleration is present in many daily activities, such as starting a bike, applying brakes, or turning a vehicle. Even objects moving at constant speed can have acceleration if their direction changes.

The concept of acceleration helps explain why forces are needed to change motion. It is closely related to Newtonâ€™s laws of motion and gravity. At the Class 10 level, acceleration is important for understanding free fall, vehicle safety, and motion graphs.

1. Acceleration is the rate of change of velocity.
2. It tells how quickly velocity changes with time.
3. Acceleration occurs when speed changes.
4. Acceleration also occurs when direction changes.
5. Even at constant speed, acceleration can exist.
6. Circular motion has acceleration due to change in direction.
7. The SI unit of acceleration is meter per second square (m/sÂ²).
8. Acceleration is

In [8]:
# chuncking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
)

In [9]:
chunks = text_splitter.split_documents(documents)

In [13]:
# investigate chunks
chunks[4].metadata

{'source': '..\\data\\concepts\\accelaration.txt'}

In [14]:
# embedding 
embedding = VertexAIEmbeddings(
    model_name="text-embedding-005")

C:\Users\DELL\AppData\Local\Temp\ipykernel_26888\933085049.py:2: DeprecationWarning: Use [`GoogleGenerativeAIEmbeddings`][langchain_google_genai.GoogleGenerativeAIEmbeddings] instead.
  embedding = VertexAIEmbeddings(
C:\Users\DELL\AppData\Local\Temp\ipykernel_26888\933085049.py:2: LangChainDeprecationWarning: The class `VertexAIEmbeddings` was deprecated in LangChain 3.2.0 and will be removed in 4.0.0. An updated version of the class exists in the `langchain-google-genai package and should be used instead. To use it run `pip install -U `langchain-google-genai` and import as `from `langchain_google_genai import GoogleGenerativeAIEmbeddings``.
  embedding = VertexAIEmbeddings(


In [15]:
# store chunks into vectors in vector store
vector_store = FAISS.from_documents(chunks, embedding)

In [16]:
retriever = vector_store.as_retriever()

In [44]:
# Now we need to take a question from user
question = "What does Accelartion describe ?"
# question = "What is heat?"
# question = input("Ask your question")

In [45]:
relevant_data = retriever.invoke(question)

In [46]:
# lets examine relevant data
print(relevant_data[0].page_content)
print(len(relevant_data))

Acceleration describes how quickly velocity changes with time. An object accelerates when it speeds
4


In [47]:
# lets make our llm a teacher for class 10 student
from langchain_core.prompts import ChatPromptTemplate

teacher_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a very helpful {class} class {subject} teacher. 
            You explain concepts in simple terms.  
            You answer questions based on the context provided. 
            If you don't know the answer, just say 'I don't know'. 
            Don't make up answers. context is {context}""",
        ),

        ("human", "{question}"),
    ]
)

In [48]:
# sample invocation of template
teacher_template.invoke(
    {
        "class": "10",
        "subject": "science",
        "context": relevant_data,
        "question": question,
    }
)


ChatPromptValue(messages=[SystemMessage(content="\n            You are a very helpful 10 class science teacher. \n            You explain concepts in simple terms.  \n            You answer questions based on the context provided. \n            If you don't know the answer, just say 'I don't know'. \n            Don't make up answers. context is [Document(id='b5e1e886-99e9-4b52-ad85-d05657f01fcb', metadata={'source': '..\\\\data\\\\concepts\\\\accelaration.txt'}, page_content='Acceleration describes how quickly velocity changes with time. An object accelerates when it speeds'), Document(id='a2fcd89b-565a-4d52-b467-90a1134b90ab', metadata={'source': '..\\\\data\\\\concepts\\\\accelaration.txt'}, page_content='The concept of acceleration helps explain why forces are needed to change motion. It is closely'), Document(id='1a8c6b76-e9c5-4c7b-8bfd-305f90517e4a', metadata={'source': '..\\\\data\\\\concepts\\\\accelaration.txt'}, page_content='when it speeds up, slows down, or changes directio

In [49]:
from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(
    model_name="gemini-2.5-flash-lite",
    temperature=0.2,
)


C:\Users\DELL\AppData\Local\Temp\ipykernel_26888\664158276.py:3: DeprecationWarning: Use [`ChatGoogleGenerativeAI`][langchain_google_genai.ChatGoogleGenerativeAI] instead.
  llm = ChatVertexAI(


In [50]:
rag_chain = teacher_template | llm

In [51]:
response = rag_chain.invoke(
    {
        "class": "10",
        "subject": "science",
        "context": relevant_data,
        "question": question,
    }
)

In [52]:
# rag response
response.pretty_print()

================================== Ai Message ==================================

Acceleration describes how quickly velocity changes with time. It's present when an object speeds up, slows down, or changes direction.


In [53]:
# direct llm response
llm.invoke(question).pretty_print()

================================== Ai Message ==================================

Acceleration describes **the rate at which an object's velocity changes over time.**

Here's a breakdown of what that means:

* **Velocity:** Velocity is a measure of both an object's **speed** and its **direction** of motion.
* **Change in Velocity:** This change can happen in a few ways:
    * **Increasing Speed:** The object is moving faster.
    * **Decreasing Speed (Deceleration):** The object is moving slower.
    * **Changing Direction:** The object is turning or altering its path, even if its speed remains constant.
* **Rate:** This refers to how *quickly* this change in velocity is occurring.

**In simpler terms, acceleration tells you how fast something is speeding up, slowing down, or changing direction.**

**Key aspects of what acceleration describes:**

* **Magnitude:** The amount of acceleration (e.g., 5 m/s²). This tells you how much the velocity changes each second.
* **Direction:** The di